In [61]:
import glob
import torch
from PIL import Image
from torch.utils.data import Dataset, DataLoader,random_split,SubsetRandomSampler, ConcatDataset
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from torchvision.utils import make_grid
from torchsummary import summary
import tqdm



In [62]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/MyDrive/challenge
!pwd


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/MyDrive/challenge
/gdrive/MyDrive/challenge


In [63]:
args={}
kwargs={}
args['batch_size']=64
args['test_batch_size']=64
args['epochs']=200  #The number of Epochs is the number of times you go through the full dataset. 
args['lr']=0.01 #Learning rate is how fast it will decend. 
args['momentum']=0.5 #SGD momentum (default: 0.5) Momentum is a moving average of our gradients (helps to keep direction).
args['seed']=1 #random seed
args['log_interval']=100 // args['batch_size']
args['cuda']=False


In [64]:
#load npy files
train_dataset = np.load('trainset.npy')
train_label = np.load('trainlabel.npy')
test_dataset = np.load('testset.npy')

print(train_dataset.shape)


(50000, 32, 32, 3)


In [65]:
train_data=[]
for idx, data in enumerate(train_dataset):
    train_data.append([data,train_label[idx]])

In [66]:
# Function: Split Data
def shuffle_train_valid_split(X, test_size=0.1, shuffle=True, random_state=1004):
    test_num = int(len(X) * test_size)
    train_num = len(X) - test_num
    
    if shuffle:
        np.random.seed(random_state)
        np.random.shuffle(X)
        X_train = X[:train_num]
        X_valid = X[train_num:]
    else:
        X_train = X[:train_num]
        X_valid = X[train_num:]
        
    return X_train, X_valid

In [67]:
train_data, valid_data = shuffle_train_valid_split(train_data)

print(len(train_data))

45000


In [68]:
# torch.manual_seed(42)


# dataset = ConcatDataset([train_data,valid_data])

# num_epochs=10
# batch_size=128
# k=10
# splits=KFold(n_splits=k,shuffle=True,random_state=42)
# foldperf={}

# print(len(train_data))
# print(len(valid_data))

In [69]:
#custom dataset 
class CustomDataset(Dataset):
    def __init__(self, input, transform=None):
        super().__init__()
        self.x_data = [data[0] for data in input]
        self.y_data = [data[1] for data in input]
        self.transform = transform 

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        x = self.x_data[idx]
        y = self.y_data[idx]
        if self.transform:
            x = self.transform(x)

        return x, y
        

In [70]:
#custom dataset 
class testDataset(Dataset):
    def __init__(self, input, transform=None):
        super().__init__()
        self.x_data =input
        self.transform = transform 

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        x = self.x_data[idx]
        if self.transform:
            x = self.transform(x)

        return x
        

In [71]:
transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(p = 0.3),
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(28,scale=(0.9,0.9)),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


In [72]:
traindata = CustomDataset(train_data, transform=transform_train)
validdata = CustomDataset(valid_data, transform=transform_test)
testdata = testDataset(test_dataset, transform=transform_test)


#data loader
train_loader = DataLoader(traindata, batch_size=args['batch_size'], shuffle=True)
valid_loader = DataLoader(validdata, batch_size=args['batch_size'], shuffle=True)
test_loader = DataLoader(testdata, batch_size=args['batch_size'], shuffle=False)


In [73]:
class Mish(nn.Module):


    def __init__(self):
        super().__init__()

    def forward(self, x):
        return x * torch.tanh(F.softplus(x))

In [74]:
import torch
import torch.nn as nn
class SEUnit(nn.Sequential):
    """ Squeeze and Excitation def.
    (hint 1): change nn.Identity() to appropriate class name
    """
    def __init__(self, ch, r=16):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(ch, int(ch/r), bias=False),
            nn.Mish(),
            nn.Linear(int(ch / r), ch, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x + y.expand_as(x)

class DepthSeperabelConv2d(nn.Module):

    def __init__(self, input_channels, output_channels, kernel_size, **kwargs):
        super().__init__()
        self.depthwise = nn.Sequential(
            nn.Conv2d(
                input_channels,
                input_channels,
                kernel_size,
                groups=input_channels,
                **kwargs),
            nn.BatchNorm2d(input_channels),
            nn.Mish()
        )

        self.pointwise = nn.Sequential(
            nn.Conv2d(input_channels, output_channels, 1),
            nn.BatchNorm2d(output_channels),
            nn.Mish()
        )

    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)

        return x


class BasicConv2d(nn.Module):

    def __init__(self, input_channels, output_channels, kernel_size, **kwargs):

        super().__init__()
        self.conv = nn.Conv2d(
            input_channels, output_channels, kernel_size, **kwargs)
        self.bn = nn.BatchNorm2d(output_channels)
        self.relu = nn.Mish()

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)

        return x


class MobileNet(nn.Module):

    def __init__(self, width_multiplier=1, class_num=100):
       super().__init__()

       alpha = width_multiplier
       self.stem = nn.Sequential(
           BasicConv2d(3, int(32 * alpha), 3, padding=1, bias=False),
           DepthSeperabelConv2d(
               int(32 * alpha),
               int(32 * alpha),
               3,
               padding=1,
               bias=False
           )
       )

       #downsample
       self.conv1 = nn.Sequential(
           DepthSeperabelConv2d(
               int(32 * alpha),
               int(32 * alpha),
               3,
               stride=1,
               padding=1,
               bias=False
           ),
           DepthSeperabelConv2d(
               int(32 * alpha),
               int(32 * alpha),
               3,
               padding=1,
               bias=False
           )
       )

       #downsample
       self.conv2 = nn.Sequential(
           DepthSeperabelConv2d(
               int(32 * alpha),
               int(32 * alpha),
               3,
               stride=1,
               padding=1,
               bias=False
           ),
           DepthSeperabelConv2d(
               int(32 * alpha),
               int(64 * alpha),
               3,
               padding=1,
               bias=False
           )
       )

       #downsample
       self.conv3 = nn.Sequential(
           DepthSeperabelConv2d(
               int(64 * alpha),
               int(64 * alpha),
               3,
               stride=1,
               padding=1,
               bias=False
           ),

           DepthSeperabelConv2d(
               int(64 * alpha),
               int(64 * alpha),
               3,
               padding=1,
               bias=False
           ),
           DepthSeperabelConv2d(
               int(64 * alpha),
               int(64 * alpha),
               3,
               padding=1,
               bias=False
           ),
           DepthSeperabelConv2d(
               int(64 * alpha),
               int(64 * alpha),
               3,
               padding=1,
               bias=False
           ),
           DepthSeperabelConv2d(
               int(64 * alpha),
               int(64 * alpha),
               3,
               padding=1,
               bias=False
           ),
           DepthSeperabelConv2d(
               int(64 * alpha),
               int(128 * alpha),
               3,
               padding=1,
               bias=False
           ),SEUnit(128)
       )

       #downsample
       self.conv4 = nn.Sequential(
           DepthSeperabelConv2d(
               int(128 * alpha),
               int(128 * alpha),
               3,
               stride=1,
               padding=1,
               bias=False
           ),SEUnit(128),
           DepthSeperabelConv2d(
               int(128 * alpha),
               int(128 * alpha),
               3,
               padding=1,
               bias=False
           ),SEUnit(128)
       )
        
       self.fc = nn.Linear(int(128 * alpha), class_num)
       self.avg = nn.AdaptiveAvgPool2d(1)
       self.pool = nn.MaxPool2d(2, 2)
       self.dropout = nn.Dropout(0.2)
      #  weight initialization with he_normal
       for m in self.modules():
        if isinstance(m, nn.Conv2d):
            nn.init.kaiming_normal_(m.weight, mode='fan_out')
        elif isinstance(m, nn.BatchNorm2d):
            m.weight.data.fill_(1)
            m.bias.data.zero_()
        

    def forward(self, x):
        x = self.stem(x)

        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        #x = self.pool(x)
        x = self.conv4(x)
        #x = self.pool(x)
        x = self.avg(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = self.fc(x)
        return x


def mobilenet(alpha=1, class_num=100):
    return MobileNet(alpha, class_num)

In [75]:
for batch, (X,y)in enumerate(train_loader):
    print(batch)
    print(X.shape)
    print(y.shape)
    break

0
torch.Size([64, 3, 28, 28])
torch.Size([64])


In [76]:
device =  "cuda:0"

In [77]:

def train_loop(dataloader, model, loss_fn, optimizer):
    model.train()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    train_loss, correct = 0, 0
    for batch, (X, y) in tqdm.tqdm(enumerate(dataloader)):
        # Compute prediction and loss
        pred = model(X.to(device))
        loss = loss_fn(pred, y.to(device))
        correct += (pred.argmax(1) == y.to(device)).type(torch.float).sum().item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step() 
        
        #print("Factor = ",i," , Learning Rate = ",optimizer.param_groups[0]["lr"])
#         if batch % 128 == 0:
#             loss, current = loss.item(), batch * len(X)
#             print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
     
    lrs.append(optimizer.param_groups[0]["lr"]) 
    print("lr :" + str(optimizer.param_groups[0]["lr"]))
    train_loss /= num_batches
    correct /= size
    print(f"Train Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {loss:>8f} \n")
    train_acc.append(correct*100)
    scheduler.step()


def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X.to(device))
            test_loss += loss_fn(pred, y.to(device)).item()
            correct += (pred.argmax(1) == y.to(device)).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    val_acc.append(100*correct)
    return correct



In [78]:

print("Using {} device".format(device))

model = mobilenet().to(device)
#model.load_state_dict(torch.load('./fixmodel3.pt'))


loss_fn = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.01,total_steps=args['epochs'],steps_per_epoch=len(train_loader), epochs=args['epochs'])
lrs = []
train_acc=[]
val_acc =[]

Using cuda:0 device


In [79]:
pytorch_total_params = sum(p.numel() for p in model.parameters())
pytorch_total_params

summary(model, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]             864
       BatchNorm2d-2           [-1, 32, 32, 32]              64
              Mish-3           [-1, 32, 32, 32]               0
       BasicConv2d-4           [-1, 32, 32, 32]               0
            Conv2d-5           [-1, 32, 32, 32]             288
       BatchNorm2d-6           [-1, 32, 32, 32]              64
              Mish-7           [-1, 32, 32, 32]               0
            Conv2d-8           [-1, 32, 32, 32]           1,056
       BatchNorm2d-9           [-1, 32, 32, 32]              64
             Mish-10           [-1, 32, 32, 32]               0
DepthSeperabelConv2d-11           [-1, 32, 32, 32]               0
           Conv2d-12           [-1, 32, 32, 32]             288
      BatchNorm2d-13           [-1, 32, 32, 32]              64
             Mish-14           [-1, 

In [ ]:

base_correct = 0
for t in range(args['epochs']):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_loader, model, loss_fn, optimizer)
#     print("test loop")
    correct = test_loop(valid_loader, model, loss_fn)
    if correct > base_correct:
        print('---------------------Model saved base correct:' + str(correct))
        torch.save(model.state_dict(), 'fixmodel_keep.pt')
        base_correct = correct
print("Done!")

Epoch 1
-------------------------------


704it [01:14,  9.49it/s]


lr :0.0003999999999999993
Train Error: 
 Accuracy: 4.6%, Avg loss: 4.095738 

Test Error: 
 Accuracy: 9.1%, Avg loss: 3.910907 

---------------------Model saved base correct:0.0906
Epoch 2
-------------------------------


704it [01:03, 11.16it/s]


lr :0.0004021898344502388
Train Error: 
 Accuracy: 9.5%, Avg loss: 3.585655 

Test Error: 
 Accuracy: 11.7%, Avg loss: 3.644189 

---------------------Model saved base correct:0.1172
Epoch 3
-------------------------------


...

704it [01:00, 11.59it/s]


lr :0.0007077932009665415
Train Error: 
 Accuracy: 77.0%, Avg loss: 1.522101 

Test Error: 
 Accuracy: 66.5%, Avg loss: 1.213780 

---------------------Model saved base correct:0.6646
Epoch 309
-------------------------------


704it [01:00, 11.62it/s]


lr :0.0006752630093736913
Train Error: 
 Accuracy: 77.3%, Avg loss: 1.824416 

Test Error: 
 Accuracy: 65.2%, Avg loss: 1.227748 

Epoch 310
-------------------------------


375it [00:32, 11.47it/s]

In [ ]:
plt.plot(train_acc)
plt.plot(val_acc)
plt.show()

In [ ]:
plt.plot(lrs)

In [ ]:
import pandas as pd
model.load_state_dict(torch.load('fixmodelkeep.pt'))
id_list = []
pred_list = []
_id = 0
model.eval()
with torch.no_grad():
    for x in test_loader:
        preds = model(x.cuda())
        preds = preds.argmax(1)
        for pred in preds:
            id_list.append(_id)
            pred_list.append(pred.item())
            _id += 1

res = pd.DataFrame({
    'id_idx': id_list,
    'label': pred_list
}) 

res.sort_values(by='id_idx', inplace=True)

res.to_csv('./Fixresultkeep.csv', index=False) 
print("result saved")

In [ ]:
!nvidia-smi --query | fgrep 'Product Name'
